In [1]:
import time
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import make_scorer
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

#datasets.load_digits()

#datasets.load_wine()

#datasets.load_breast_cancer()

In [ ]:
# ZeroR

clf = DummyClassifier(strategy='most_frequent')
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.00034618, 0.00040722, 0.0001564 , 0.00014687, 0.00014472,
        0.00019336, 0.00017929, 0.00014949, 0.00013137, 0.00021029]),
 'score_time': array([0.00048065, 0.00025129, 0.00020885, 0.00021744, 0.00020885,
        0.00020218, 0.00018573, 0.00017905, 0.00019026, 0.00019717]),
 'test_score': array([0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333])}

In [ ]:
# Aleatório

clf = DummyClassifier(strategy='uniform')
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.00029922, 0.00034571, 0.00023222, 0.00023699, 0.00025082,
        0.00024247, 0.00022244, 0.00022936, 0.00021744, 0.00019503]),
 'score_time': array([0.00073671, 0.00042367, 0.00037861, 0.00068903, 0.00058055,
        0.00036216, 0.00033784, 0.00035858, 0.00051355, 0.00020409]),
 'test_score': array([0.26666667, 0.6       , 0.4       , 0.2       , 0.06666667,
        0.46666667, 0.26666667, 0.2       , 0.46666667, 0.13333333])}

In [ ]:
# Aleatório Estratificado

clf = DummyClassifier(strategy='stratified')
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.000314  , 0.00030065, 0.00027132, 0.00029349, 0.00044155,
        0.00024605, 0.00024509, 0.00025249, 0.00043106, 0.0001421 ]),
 'score_time': array([0.00086737, 0.00052023, 0.00067925, 0.00043964, 0.00042653,
        0.00037956, 0.00063419, 0.00039887, 0.00030518, 0.00023174]),
 'test_score': array([0.4       , 0.46666667, 0.6       , 0.46666667, 0.46666667,
        0.4       , 0.2       , 0.4       , 0.53333333, 0.2       ])}

In [3]:
# OneR Probabilístico
'''OneR Algorithm		
For each predictor,
     For each value of that predictor, make a rule as follows;
           Count how often each value of target (class) appears
           Find the most frequent class
           Make the rule assign that class to this value of the predictor
     Calculate the total error of the rules of each predictor
Choose the predictor with the smallest total error.
'''

class OneRProbabilistic(BaseEstimator, ClassifierMixin):
    def __init__(self, disc=True):
        self._disc = disc

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = np.unique(y)
        self.n_classes_ = len(self.classes_)

        self.X_ = X
        self.y_ = y

        # Discretization
        self.__est = KBinsDiscretizer(n_bins=2*self.n_classes_, encode='ordinal', strategy='kmeans') if self._disc else None
        X_disc = self.__est.fit_transform(X) if self._disc else X

        # Contingency tables
        contingency_tb = [pd.crosstab(x, y, rownames=['data'], colnames=['target']) for x in X_disc.T]
        # Feature with greater power differentiation
        self.__great_diff = np.argmax([tb.max(axis=1).sum() for tb in contingency_tb])
        # Class distribution of each characteristic value
        self.__dist_class = contingency_tb[self.__great_diff].div(contingency_tb[self.__great_diff].sum(axis=1), axis=0)

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Discretization
        X_disc = self.__est.transform(X) if self._disc else X

        # Input validation
        X_disc = check_array(X_disc)

        # roulette taking into account the distribution of classes
        predicted = [self.__dist_class.columns[(self.__dist_class.loc[x].cumsum() >= np.random.rand()).idxmax()] for x in X_disc[:,self.__great_diff]]
        return np.array(predicted)
    
    def get_params(self, deep=False):
        return {'disc': self._disc}

def my_score(y_true, y_pred):
    diff = np.abs(y_true - y_pred).max()
    return np.log1p(diff)

cls = OneRProbabilistic()
cross_validate(cls, X, y, cv=10, scoring=make_scorer(my_score, greater_is_better=False))

{'fit_time': array([0.06992817, 0.06176043, 0.06120229, 0.06170654, 0.05910063,
        0.05792117, 0.05621099, 0.05675745, 0.06104231, 0.05714488]),
 'score_time': array([0.00870419, 0.00777602, 0.00920272, 0.00799513, 0.00757718,
        0.00772285, 0.00776792, 0.00798035, 0.00976801, 0.00778103]),
 'test_score': array([-0.69314718, -0.69314718, -0.69314718, -0.        , -0.69314718,
        -0.69314718, -0.69314718, -0.        , -0.        , -0.        ])}

In [ ]:
# Naive Bayes Gaussiano

clf = GaussianNB()
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.00377393, 0.00157189, 0.00063181, 0.00053978, 0.00052428,
        0.0005281 , 0.00052261, 0.0005784 , 0.00054383, 0.00053453]),
 'score_time': array([0.00112891, 0.00041986, 0.00036001, 0.00034404, 0.00040078,
        0.00038052, 0.00038886, 0.00033689, 0.00032496, 0.00033402]),
 'test_score': array([0.93333333, 0.93333333, 1.        , 0.93333333, 0.93333333,
        0.93333333, 0.86666667, 1.        , 1.        , 1.        ])}

In [4]:
# KCentroids

'''KCentroids
The KCentroides classifier uses a grouping algorithm to define K groups of examples from each class in the training base.
Assuming that a database has ncl classes, the KCentroides algorithm initially forms K * ncl groups, with K groups in each of the ncl classes.
Then, the centroid of each group is calculated and this centroid is associated with the class of the group from which it was generated.
The hyperparameter method has the value of K.
To perform a classification, KCentroides checks which centroid is closest to the element to be classified and returns to its class.
'''

class KCentroids(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator):
        self._estimator = estimator

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = np.unique(y)

        self.X_ = X
        self.y_ = y

        # Geting centroids with estimator
        self.__groups = [self._estimator.fit(X[y == c]).cluster_centers_ for c in self.classes_]

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        predicted = [self.classes_[np.argmin([min([np.linalg.norm(x - c) for c in g]) for g in self.__groups])] for x in X]
        return np.array(predicted)

    def get_params(self, deep=False):
        return {'estimator': self._estimator}

In [5]:
# KmeansCentroides

clf = KCentroids(KMeans(n_clusters=5))
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.14534688, 0.09378791, 0.09419107, 0.09767652, 0.09479761,
        0.10348368, 0.0956831 , 0.09007311, 0.11101365, 0.09948659]),
 'score_time': array([0.00211143, 0.00208426, 0.00211573, 0.00211835, 0.00208163,
        0.00208926, 0.00216889, 0.00382209, 0.0022068 , 0.0028441 ]),
 'test_score': array([1.        , 0.93333333, 1.        , 1.        , 0.93333333,
        0.93333333, 0.93333333, 1.        , 1.        , 1.        ])}

In [15]:
# KGACentroides

class AG():
'''Genetic Algorithm
    The initial population is constructed by hill climbing algorithm. 
    The selection mechanism is given by tournament, where a number of candidates for parents are put forward and only the best two are selected to generate offspring.
    Since the crossing mechanism selects only two individuals, this step is repeated pop_size/2 times, so that all individuals have an equal chance of performing the recombination.
    The combination mechanism used was also highly inspired by nature, to which the father donates a sperm that will be fertilized in an egg donated by the mother.
    Technically, the father donates a randomly chosen cluster and it is inserted into the mother. 
    Then, the items present in the cluster donated by the father are removed from the mother cluster generating a new individual. 
    If it occurs to result in the child more clusters than necessary, those with the least amount of items are grouped; otherwise, the cluster with the largest amount are separated in half.
    This technique was adopted so that good groupings can be maintained.
    The mutation mechanism applied is strong: if the individual is selected, he is discarded and a new individual is inserted into the population.
    Note, therefore, that the mutation occurs more in the population than in the individual.
    As for the offspring, only the pop_size best individuals are allowed to enter a new generation.
    As a stopping criterion, maximum number of iterations, maximum processing time and whether the population has converged, that is, if all individuals are identical, are used.
'''
    def __init__(self, k, pop_size, iter_max, cross_ratio, mut_ratio, max_time=1):
        self.k_ = k
        self.pop_size_ = pop_size
        self.iter_max_ = iter_max
        self.cross_ratio_ = cross_ratio
        self.mut_ratio_ = mut_ratio
        self.max_time_ = max_time

    def fit(self, X):
        # Check that X have correct shape
        X = check_array(X)

        self.X_ = X

        # Geting centroids with estimator
        solution = self.genetic(X, self.k_, self.pop_size_, self.iter_max_, 
                                       self.cross_ratio_, self.mut_ratio_, self.max_time_)
        self.cluster_centers_ = solution['cluster_centers_'],
        self.sse_ = solution['sse_'],
        self.clusters_ = solution['clusters_']

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        predicted = [self.classes_[np.argmin([np.linalg.norm(x - g) for g in self.__solution['cluster_centers_']])] for x in X]
        return np.array(predicted)


    def __available_closest_items(self, cluster, available_items, n):
        # n items more closet to centroid of cluster
        n = min(n, len(available_items))
        evaluations = [self.__evaluate_cluster(cluster + [i]) for i in available_items]
        closest = np.argpartition([e['sum_dist'] for e in evaluations], range(n))[:n]
        return closest

    def hill_climbing(self, items, k):
        num_best = k
        available_items = list(range(len(items)))
        np.random.shuffle(available_items)
        # choose the firsts items of clusters randomly
        clusters = [[items[available_items.pop()]] for _ in range(k)]

        current_cluster = 0
        while available_items:
            closest = self.__available_closest_items(clusters[current_cluster], [items[p] for p in available_items], num_best)
            choiced = available_items[closest[np.random.randint(len(closest))]]
            clusters[current_cluster].append(items[choiced])
            available_items.remove(choiced)
            current_cluster = (current_cluster + 1) % k

        return clusters

    def __evaluate_cluster(self, cluster):
        # euclidean norm
        items = [s['coord'] for s in cluster]
        mu = np.average(items, axis=0) # centroid
        
        norm2 = [np.linalg.norm(i-mu) for i in items]
        return {'sum_dist': sum(norm2), 'dist': norm2, 'mu': mu.tolist(), 'items': cluster}

    def __evaluate_clusters(self, clusters):
        states = [self.__evaluate_cluster(cluster) for cluster in clusters]
        return states

    def __objective_function(self, states):
        # SSE metric
        sse = sum([state['sum_dist'] for state in states])
        return sse

    def __random_state(self, states):
        n = len(states)-1
        if n <= 1:
            return states[0]

        index = np.random.randint(0, n)
        return states[index]

    def __initial_population(self, n, items, k):
        pop = [self.__evaluate_clusters(self.hill_climbing(items, k)) for _ in range(n)]
        return pop

    def __selection(self, population, n, n_tournament):
        # by tournament
        N = len(population)
        if n >= N:
            return population
        if n_tournament > N:
            n_tournament = N

        selecteds = np.random.choice(range(0, len(population)), size=n_tournament, replace=False)
        selecteds.sort()
        values = [self.__objective_function(population[i]) for i in selecteds]
        pos_ordered = np.argpartition(values, range(n))[-n:]
        return [population[p] for p in pos_ordered]

    def __crossover(self, dad, mom):
        # son product of an egg fertilized by a sperm
        sperm = self.__random_state(dad)['items']
        # take all groups that not contain the sperm, and remove it too
        egg = [e for e in [[n for n in m['items'] if n not in sperm] for m in mom] if e]
        son = egg + [sperm]

        k = len(dad)
        while len(son) > k: # concatenate smaller groups
            smallers = np.argpartition([len(s) for s in son], range(2))[:2]
            son[smallers[0]] += son[smallers[1]]
            del son[smallers[1]]

        while len(son) < k: # separate larger groups
            larger = np.argmax([len(s) for s in son])
            half = len(son[larger])//2
            son += [son[larger][:half]] + [son[larger][half:]]
            del son[larger]

        return self.__evaluate_clusters(son)

    def __mutation(self, items, k):
        # new individual
        return self.__evaluate_clusters(self.hill_climbing(items, k))

    def __convergent(self, population):
        clusters_0 = [sorted([i['id'] for i in p['items']]) for p in population[0]]
        clusters = [[sorted([i['id'] for i in clusters['items']]) for clusters in states] for states in population]
        return all(c == clusters_0 for c in clusters)

    def __evaluate_population(self, population):
        return sum([self.__objective_function(p) for p in population], [])

    def __offspring(self, population, n):
        best_index = np.argpartition([sum([q['sum_dist'] for q in p]) for p in population], range(n))[:n]
        return [population[i] for i in best_index]

    def genetic(self, X, k, pop_size, iter_max, cross_ratio, mut_ratio, max_time):
        items = [{'id': x, 'coord': y} for x, y in zip(range(len(X)), X)]
        n_tournament = 3
        half_pop = pop_size//2
        pop = self.__initial_population(pop_size, items, k)
        iter = 0
        end = 0

        start = time.process_time()
        while True:
            new_pop = pop.copy()
            for _ in range(half_pop): # everyone can cross
                if np.random.uniform(0, 1, 1) <= cross_ratio:
                    parents = self.__selection(pop, 2, n_tournament)
                    new_pop.append(self.__crossover(parents[0], parents[1]))
                if np.random.uniform(0, 1, 1) <= mut_ratio:
                    new_pop.append(self.__mutation(items, k))
            pop = self.__offspring(new_pop, pop_size)
            val_pop = self.__evaluate_population(pop)

            iter += 1
            end = time.process_time()
            if iter >= iter_max:
                br = 'iteration'
                break
            if end-start > max_time:
                br = 'time'
                break
            if self.__convergent(pop):
                br = 'convergence'
                break

        best_individual = self.__offspring(pop, 1)[0]
        return {'cluster_centers_': [b['mu'] for b in best_individual],
                'sse_': [b['sum_dist'] for b in best_individual],
                'clusters_': [[c['coord'] for c in b['items']] for b in best_individual]}


clf = KCentroids(AG(3, 50, 100, 0.8, 0.1, 1))
cross_validate(clf, X, y, cv=10, scoring=make_scorer(my_score, greater_is_better=False))

{'fit_time': array([12.56269455, 12.61457133, 12.41115999, 12.45732808, 12.37376428,
        12.4879539 , 12.6698463 , 12.57104206, 12.55874062, 12.512784  ]),
 'score_time': array([0.0010848 , 0.00107121, 0.00118184, 0.00097609, 0.00107002,
        0.0010891 , 0.00107265, 0.00099301, 0.00108051, 0.00109649]),
 'test_score': array([-0.69314718, -0.69314718, -0.69314718, -0.69314718, -0.69314718,
        -0.69314718, -0.        , -0.69314718, -0.69314718, -0.69314718])}

In [ ]:
# Knn

clf = KNeighborsClassifier(n_neighbors=5, weights='uniform')
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.00089765, 0.000525  , 0.00052905, 0.00051475, 0.00047994,
        0.00049734, 0.00048304, 0.00046825, 0.00051212, 0.00047445]),
 'score_time': array([0.00173497, 0.00153756, 0.00147939, 0.00145245, 0.00145221,
        0.00139809, 0.00182414, 0.00138259, 0.00136447, 0.00136137]),
 'test_score': array([1.        , 0.93333333, 1.        , 1.        , 0.86666667,
        0.93333333, 0.93333333, 1.        , 1.        , 1.        ])}

In [ ]:
# DistKnn

clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.00063038, 0.00055408, 0.00052285, 0.00051045, 0.00048232,
        0.00048113, 0.00046873, 0.00049949, 0.00052381, 0.00045943]),
 'score_time': array([0.00147986, 0.00115657, 0.00104284, 0.00104022, 0.00096202,
        0.00094938, 0.00094032, 0.0009234 , 0.00095439, 0.00094771]),
 'test_score': array([1.        , 0.93333333, 1.        , 1.        , 0.86666667,
        0.93333333, 0.93333333, 1.        , 1.        , 1.        ])}

In [ ]:
# Árvore de Decisão

clf = DecisionTreeClassifier(max_depth=None)
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.00125885, 0.0066998 , 0.00059628, 0.00048113, 0.00042081,
        0.00043058, 0.00041461, 0.00042415, 0.00041509, 0.00043321]),
 'score_time': array([0.00072908, 0.00063658, 0.00031853, 0.00026727, 0.00025368,
        0.00026298, 0.0002532 , 0.00024772, 0.00024533, 0.0002737 ]),
 'test_score': array([1.        , 0.93333333, 1.        , 0.93333333, 0.93333333,
        0.86666667, 0.93333333, 0.93333333, 1.        , 1.        ])}

In [ ]:
# Florestas de Árvores

clf = RandomForestClassifier(n_estimators=10)
cross_validate(clf, X, y, cv=10)

{'fit_time': array([0.01583815, 0.01362228, 0.01345134, 0.01485443, 0.01329064,
        0.01354814, 0.01352906, 0.01733541, 0.0138247 , 0.01379824]),
 'score_time': array([0.00143909, 0.0014596 , 0.00137806, 0.00139523, 0.0013907 ,
        0.00138092, 0.001369  , 0.00140595, 0.00139141, 0.00138974]),
 'test_score': array([1.        , 0.93333333, 1.        , 0.93333333, 0.93333333,
        0.86666667, 0.93333333, 1.        , 1.        , 1.        ])}